### File to work on the training process for self supervised learning

In [2]:
import extract_colmap as c
import numpy as np
import torch 
import os
import cv2
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torchvision.transforms as T
from PIL import Image

#ghost_city_bin_path='raw/test/bin_files/ghost_city_bins/images.bin'
#medium_structure_bin_path='raw/test/bin_files/medium_structure_bins/images.bin'

### IMPORTANT PATHS
ghost_city_bin_path='/home/server/Ines/models/raw/test/bin_files/ghost_city_bins/images.bin'
medium_structure_bin_path='/home/server/Ines/models/raw/test/bin_files/medium_structure_bins/images.bin'
thermitiere_bin_path='/home/server/Ines/models/raw/train/bin_files/thermitiere_bins/images.bin'
old_cliff_bin_path='/home/server/Ines/models/raw/train/bin_files/old_rainbow_cliff_bins/images.bin'

In [5]:
# From the quaternion to the rotation matrix:

def quaternion_rotation_matrix(Q):
    """
    Coverts a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """

    # Check the quaternion size:
    if len(Q)!=4:
        return 'SizeError: A quaternion contains 4 values'
    
    # Extract the values from Q
    q0 = Q[0]
    q1 = Q[1]
    q2 = Q[2]
    q3 = Q[3]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

In [8]:
print(quaternion_rotation_matrix((2.1,3,4,5)))

[[25.82  3.   46.8 ]
 [45.   39.82 27.4 ]
 [13.2  52.6  57.82]]


In [12]:
# Form the reference change matrix

def get_T_matrix(qvec, tvec):
    if len(qvec)!=4:
        return 'SizeError: A quaternion contains 4 values'

    if len(tvec)!=3:
        return 'SizeError: A translation contains 3 values'
    
    # Extract the values from Q
    q0 = qvec[0]
    q1 = qvec[1]
    q2 = qvec[2]
    q3 = qvec[3]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])

    # Full T_world-->cam matrix
    T=np.array([[r00, r01, r02, tvec[0]],
                [r10, r11, r12, tvec[1]],
                [r20, r21, r22, tvec[2]],
                ])
    return T


In [13]:
q_test=(1,2,3,4)
t_test=(5,6,7)
print(get_T_matrix(q_test, t_test))

[[ 9  4 22  5]
 [20 19 20  6]
 [10 28 33  7]]


In [34]:
# Reprojection error between two images 

def get_reproj_error(I1, I2, T1, T2, K, d1):
    """
    Inputs: A sequential set of 2 images (I1, I2)
            Their associated pose camera (from world to the camera pose) (T1, T2) (4*4 matrix)
            The camera calibration matrix K
            The predicted depth map d1
    Output: The reprojection error between the 2 images  R_1-->2
    """
    error=0
    H,W=np.shape(I1)

    # Changing references
    T2_inv=np.linalg.pinv(T2)
    K_inv=np.linalg.inv(K)    
  

     # Checking shapes 

    print('T2 shape', np.shape(T2))
    print('T1 shape', np.shape(T1))
    print('T2_inv shape', np.shape(T2_inv))
    print('K_inv shape', np.shape(K_inv))

    # Bruh
    G1= np.transpose(T1)@ K_inv
    G1= np.transpose(T2_inv) @ G1
    G1= K @ G1

    # R_1-->2
    for u in range(H):
        for v in range(W):
                p1=np.array([u, v, 1])
                p2=np.array(G1 @ (d1[u,v]*p1))
                delta=p1-p2
                # Euclidean distance
                error+=np.sqrt(delta[0]**2+delta[1]**2)
                
    return error




In [37]:
## DUMMY VALUES 

I1=np.ones((5,5))
I2=3*np.ones((5,5))
K=np.array([[1, 0, 0.3],
            [0, 1, 0.2],
            [0, 0,   1],
            ])

T1= np.array([[ 9, 4, 22,  5],
 [20, 19, 20,  6],
 [10, 28, 33,  7]])

T2= np.array([[ 9, 4, 12,  5],
 [20, 3, 20,  6],
 [10, 28, 6,  7]])

d1=np.ones((5,5))

d2=4*np.ones((5,5))

err_12=get_reproj_error(I1, I2, T1, T2, K, d1)
err_21=get_reproj_error(I2, I1, T1, T2, K, d2)
print(err_12)
print(err_21)



T2 shape (3, 4)
T1 shape (3, 4)
T2_inv shape (4, 3)
K_inv shape (3, 3)
T2 shape (3, 4)
T1 shape (3, 4)
T2_inv shape (4, 3)
K_inv shape (3, 3)
627.9614489861225
1029.6451936835952


In [15]:
# How it is going to look like in the trainig process. 
train_loader=None

for image, T in train_loader:
    # Reminder= image, T are batches of batch_size elements

    #Moving to GPU 
    image.to(device)
    T.to(device)

    #Right type
    image_batch=image.type(torch.cuda.FloatTensor)
    # T = np.array

    # Going through the network
    



(3, 4)
